___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# [멀티쓰레딩(Multithreading)](https://docs.python.org/3/library/threading.html)


### 기본적인 사용 방법

사용 가능한 코어 수 확인

In [ ]:
import os

os.cpu_count()

쓰레드 생성, 시작, 대기

In [ ]:
import threading
import os

def thread_work(message):
    print(threading.current_thread().name) # 현재 이 함수를 실행하고 있는 쓰레드 이름
    print("Process ID", os.getpid()) # 프로세스 아이디
    print(message)

t1 = threading.Thread(target=thread_work, args=("First thread",)) # 마지막 컴마로 튜플 넘겨주기
t2 = threading.Thread(target=thread_work, args=("Second thread",)) # 마지막 컴마로 튜플 넘겨주기

t1.start()
t2.start()

t1.join()
t2.join()


### Race Condition

- 여러 쓰레드들이 지역 변수는 따로따로 만들어 사용하지만 전역 변수는 같이 사용합니다. 
- 일반적으로 여러 쓰레드가 동시에 같은 메모리의 값을 바꾸려고 시도하면 문제가 생길 수 있습니다. 
- 문제가 항상 생기는 것이 아니기 때문에 디버깅이 어렵습니다.
```
첫 번째 쓰레드가 g의 현재 값(0)을 읽어옴
두 번째 쓰레드가 g의 현재 값(0)을 읽어옴
첫 번째 쓰레드가 0 + 10을 g에 저장해서 g가 10
두 번째 쓰레드가 0 + 10을 g에 저장해서 g가 10
결과적으로 g가 10 (우리가 기대한 값은 20)
```
- CPython은 GIL이 race condition을 막아주지만 절대로 GIL을 이용하는 방식으로 습관을 들이면 안됩니다.


In [ ]:
import threading

g = 0

def thread_work(message):
    global g

    l = len(message) # 지역 변수
    print(threading.current_thread().name, id(l), id(g))

    g = g + 10 # Race condition! Bad!!!

t1 = threading.Thread(target=thread_work, args=("First thread",)) # 마지막 컴마로 튜플 넘겨주기
t2 = threading.Thread(target=thread_work, args=("Second thread",)) # 마지막 컴마로 튜플 넘겨주기

t1.start()
t2.start()

t1.join()
t2.join()

print(g) # 20


### Lock

여러 쓰레드가 함께 사용하는 메모리에 쓰기 작업을 할 때는 잠금장치(Lock)를 걸어줄 수 있습니다. 그러나 Lock을 사용한다는 것은 알고리즘을 동기식(synchronous)으로 바꾸는 것이기 때문에 효율에 매우 안좋습니다.

In [ ]:
import threading
import os

g = 0

lock = threading.Lock()

def thread_work(message):
    global g

    l = len(message) # 지역 변수
    print(threading.current_thread().name, id(l), id(g))

    lock.acquire()
    g = g + 10
    lock.release()

t1 = threading.Thread(target=thread_work, args=("First thread",)) # 마지막 컴마로 튜플 넘겨주기
t2 = threading.Thread(target=thread_work, args=("Second thread",)) # 마지막 컴마로 튜플 넘겨주기

t1.start()
t2.start()

t1.join()
t2.join()

print(g) # 20


### IO-Bound  문제 예시

앞에서 만들었던 타이머 사용 (사용 전 셀 실행)

In [2]:
import time

class Timer:
    def __init__(self):
        self.start = time.time()

    def __enter__(self):
        return self # as로 사용할 수 있도록 self 반환

    def __exit__(self, *args):
        print("Elapsed time = ", time.time() - self.start)

한 번에 하나씩 직렬(Serial)로 실행할 경우

In [3]:
import time

def thread_work():
    time.sleep(0.1) # IO 대기 시간이 0.1초라고 가정

num_tasks = 10

with Timer():
    for i in range(num_tasks):
        thread_work()

Elapsed time =  1.0711345672607422


멀티쓰레딩으로 동시성(Concurrency) 구현

In [6]:
import time
import threading

def thread_work():
    time.sleep(0.1) # IO 대기 시간이 0.1초라고 가정
    #print(threading.current_thread().name)

num_tasks = 10

with Timer():
    thread_list = []
    for _ in range(num_tasks):
        t = threading.Thread(target=thread_work, args=())
        t.start()
        thread_list.append(t)

    for t in thread_list:
        t.join()

Elapsed time =  0.11521577835083008


[주의] join 위치

In [ ]:

import time
import threading

def thread_work():
    time.sleep(0.1) # IO 대기 시간이 0.1초라고 가정

num_tasks = 10

with Timer():
    for i in range(num_tasks):
        t = threading.Thread(target=thread_work, args=())
        t.start()
        t.join() # 시리얼로 처리

### [실습] 온도가 가장 높은 도시 찾기

앞에서 사용했던 현재 날씨 데이터 받기

In [ ]:
import requests

API_KEY = "YOUR_KEY_HERE"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
LANGUAGE = "kr" # 출력 중 Weather에 한국어로 받을 수 있음. 영어를 원하면 "en"이나 생략

city = input("Enter a city name: ")
request_url = f"{BASE_URL}?appid={API_KEY}&q={city}&lang={LANGUAGE}"

print(request_url)

response = requests.get(request_url) # 웹브라우저에 똑같이 사용

if response.status_code == 200: # HTTP status 200은 성공을 의미합니다.

    data = response.json()
    city_name = data['name']
    weather = data['weather'][0]['description']
    temperature = round(data["main"]["temp"] - 273.15, 2) # 켈빈 온도 사용

    print("City:", city_name)
    print("Weather:", weather)
    print("Temperature:", temperature, "celsius")

else:
    print("An error occurred.", response.status_code)

멀티쓰레딩을 이용해서 온도가 가장 높은 도시 이름을 출력해봅시다.

```
cities = ["Seoul", "Pusan", "Paris", "London", "Cairo", "Lagos", "Giza", "Los Angeles", "San Francisco", "New York"]
```

주의 사항
- 인터넷 서비스를 남용할 경우 서버에서 막아버릴 수도 있습니다.
- 인터넷 속도, 서버와의 통신 상태 등에 따라서 결과가 달라집니다.

In [ ]:
import time

class Timer:
    def __init__(self):
        self.start = time.time()

    def __enter__(self):
        return self # as로 사용할 수 있도록 self 반환

    def __exit__(self, *args):
        print("Elapsed time = ", time.time() - self.start)